# Set Cover Problem

### Imports

In [1]:
import time

import sys
from collections import namedtuple
import math
from gurobipy import *

### Gurobi variables

In [2]:
# time limit in seconds
time_limit = 1000
# memory limit in bytes
memory_limit = 1000 ** 9

DEBUG = 2


### Classes

In [3]:
class Subset:
        def __init__(self, index, weight, elementList):   #construttor
                self.index = index
                self.weight = weight
                self.elementList = elementList
                    
class Element:
        def __init__(self, index, subsets):   #construttor
                self.index = index
                self.subsets = subsets

### Set Cover LP implementation

In [4]:
def setCoverLP(subsetList, elementList):
    #Instantiate Gurobi Solver
    
    solver = Model('SolveLinearProblem')
    
    #Turn verbose off
    solver.setParam(GRB.Param.OutputFlag, 0)
    
    # Set a time limit
    solver.setParam(GRB.Param.TimeLimit, time_limit)
                  
    # Creating decision variable
    x = []
    for j in range(0, len(subsetList)):
        x.append(solver.addVar(vtype=GRB.CONTINUOUS, name='x[%d]' % j))
                    
    # Xj >= 1, i = 1, ..., n
    # i/n elements j/m subsets
    for i in range(0, len(elementList)):
        left_side = LinExpr()
        for j in range(0, len(elementList[i].subsets)):
            left_side += 1* x[elementList[i].subsets[j]]
        solver.addConstr(left_side, GRB.GREATER_EQUAL, 1, 'c1[%d]' % j)
        
    # Xj >= 0, j = 1, ..., m
    for j in range(0, len(subsetList)):
        solver.addConstr(x[j], GRB.GREATER_EQUAL, 0, 'c2[%d]' % j)
    
    # Objective function
    solver.setObjective(
        (quicksum(subsetList[j].weight * x[j]
                 for j in range(0, len(subsetList)))
        ), GRB.MINIMIZE
    )
    
    #Solves the LP Model
    solver.optimize()
        
    if DEBUG >= 2:
        print("Result status = {solver.status}")

    intOPT = 0
    if solver.SolCount == 0:

        if DEBUG >= 2:
            print("LP model does not found a solution")

        return ([], intOPT)

    # The problem has an optimal solution.
    if solver.status == GRB.Status.OPTIMAL:
        intOPT = 1

        if DEBUG >= 2:
            print("LP solver found optimal solution")

    elif solver.status == GRB.Status.TIME_LIMIT:

        if DEBUG >= 2:
            print("LP solver ended due to time limit")

    elif solver.status == GRB.Status.SUBOPTIMAL:

        if DEBUG >= 2:
            print("LP solver found feasible solution")

    elif solver.status == GRB.Status.INF_OR_UNBD:

        if DEBUG >= 2:
            print("LP model is infeasible or unbounded")

        return ([], 0)

    elif solver.status == GRB.Status.CUTOFF:

        if DEBUG >= 2:
            print("LP model solution is worse than bound")

        return ([], 0)

    elif solver.status == GRB.Status.LOADED:

        if DEBUG >= 2:
            print("LP solver does not found a solution")

        return ([], 0)

    else:
        print("Weird return from LP solver")
        exit(1)

    if DEBUG >= 2:
        # The objective value of the solution.
        print("Optimal objective value = %.2f" % solver.objVal)

    #Solution value
    sol_value = solver.objVal

    solution = []
    
    for subset in x:
        solution.append(subset.getAttr("x"))

    if DEBUG >= 3:
        print(solution)

    return (sol_value, solution, intOPT)
    
    return 0


    

In [6]:
inputFather = open('testCases.txt', 'r')
line = inputFather.readline()
while(line):
    line = line.strip()
    inputFileName = "setCoverTestCases/" + line
    inputFile = open(inputFileName, 'r')
    n, m = map(int,inputFile.readline().split())
    elementsList = [Element(-1, []) for x in range(n)]
    subsetsList = [Subset(x, -1, []) for x in range(m)]
    subsetWeights = []
    subsetCount = 0
    subsetsCounter = 0
    
    #N Elementos, M Subconjuntos    
    while(subsetCount < m):
        fileLine = inputFile.readline().split()
        subsetCount += len(fileLine)
        for j in range(len(fileLine)):
            subsetsList[subsetsCounter].weight = int(fileLine[j])
            subsetsCounter += 1
    
    for i in range(n):
        numberLine = int(inputFile.readline())
        subsetsAmount = []
        
        while(len(subsetsAmount) < numberLine):
            subsetsAmount += inputFile.readline().split()
        
        elementsList[i].index = i
        subsetsAmount = map(lambda x: int(x)-1, subsetsAmount)
        elementsList[i].subsets = subsetsAmount
        
        for subset in subsetsAmount:
            subsetsList[subset].elementList.append(i)
            
    inputFile.close()
    
    outputFileName = "Results/" + line
    outputFile = open(outputFileName, 'w')
    
    print(outputFileName)
    LPCost, LPSubsets, intOptFound = setCoverLP(subsetsList, elementsList)
    
    outputFile.write(repr(LPCost) + '\n')
    for subset in LPSubsets:
        outputFile.write(repr("%.4f" % subset) + '\n')
    outputFile.close()
    line = inputFather.readline()
inputFather.close()

Results/scp41.txt
Result status = {solver.status}
LP solver found optimal solution
Optimal objective value = 429.00
Results/scp410.txt
Result status = {solver.status}
LP solver found optimal solution
Optimal objective value = 513.50
Results/scp42.txt
Result status = {solver.status}
LP solver found optimal solution
Optimal objective value = 512.00
Results/scp43.txt
Result status = {solver.status}
LP solver found optimal solution
Optimal objective value = 516.00
Results/scp44.txt
Result status = {solver.status}
LP solver found optimal solution
Optimal objective value = 494.00
Results/scp45.txt
Result status = {solver.status}
LP solver found optimal solution
Optimal objective value = 512.00
Results/scp46.txt
Result status = {solver.status}
LP solver found optimal solution
Optimal objective value = 557.25
Results/scp47.txt
Result status = {solver.status}
LP solver found optimal solution
Optimal objective value = 430.00
Results/scp48.txt
Result status = {solver.status}
LP solver found optim

Results/scpnrg2.txt
Result status = {solver.status}
LP solver found optimal solution
Optimal objective value = 142.07
Results/scpnrg3.txt
Result status = {solver.status}
LP solver found optimal solution
Optimal objective value = 148.27
Results/scpnrg4.txt
Result status = {solver.status}
LP solver found optimal solution
Optimal objective value = 148.95
Results/scpnrg5.txt
Result status = {solver.status}
LP solver found optimal solution
Optimal objective value = 148.23
Results/scpnrh1.txt
Result status = {solver.status}
LP solver found optimal solution
Optimal objective value = 48.12
Results/scpnrh2.txt
Result status = {solver.status}
LP solver found optimal solution
Optimal objective value = 48.64
Results/scpnrh3.txt
Result status = {solver.status}
LP solver found optimal solution
Optimal objective value = 45.20
Results/scpnrh4.txt
Result status = {solver.status}
LP solver found optimal solution
Optimal objective value = 44.04
Results/scpnrh5.txt
Result status = {solver.status}
LP solve